In [72]:
# Import

import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import json
import re
import os
import networkx as nx

In [73]:
def load_jsonc(filepath):
    """Load JSONC file (JSON with comments)"""
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Remove single-line comments (// ...)
    content = re.sub(r'//.*?$', '', content, flags=re.MULTILINE)
    
    # Remove multi-line comments (/* ... */)
    content = re.sub(r'/\*.*?\*/', '', content, flags=re.DOTALL)
    
    # Remove trailing commas before closing brackets/braces
    content = re.sub(r',\s*([}\]])', r'\1', content)
    
    return json.loads(content)

def load_json(filepath):
    """Load regular JSON file"""
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

def load_any(filepath):
    """Load either JSON or JSONC file based on extension"""
    if filepath.endswith('.jsonc'):
        return load_jsonc(filepath)
    else:
        return load_json(filepath)

In [74]:
# TASK 1: relocate age_weights to parameters.jsonc file
input_dir = './input_data'
stage_dirs = ['amd_sim_data', 'amd_sim_data_Early', 'amd_sim_data_Intermediate', 'amd_sim_data_Late-dry', 'amd_sim_data_Late-wet']

for stage_dir in stage_dirs:
    parameters = load_jsonc(f'{input_dir}/{stage_dir}/parameters.jsonc')
    print(f'{stage_dir:<30} parameters keys: {parameters.keys()}')

amd_sim_data                   parameters keys: dict_keys(['rr', 'f', 'i', 'age_weights', 'ages', 'p', 'r', 'pf', 'X'])
amd_sim_data_Early             parameters keys: dict_keys(['rr', 'f', 'i', 'age_weights', 'ages', 'p', 'r', 'pf', 'X'])
amd_sim_data_Intermediate      parameters keys: dict_keys(['rr', 'f', 'i', 'age_weights', 'ages', 'p', 'r', 'pf', 'X'])
amd_sim_data_Late-dry          parameters keys: dict_keys(['rr', 'f', 'i', 'age_weights', 'ages', 'p', 'r', 'pf', 'X'])
amd_sim_data_Late-wet          parameters keys: dict_keys(['rr', 'f', 'i', 'age_weights', 'ages', 'p', 'r', 'pf', 'X'])


In [75]:
# TASK 2: update Load based on region_id_graph
stage_dirs = ['amd_sim_data', 'amd_sim_data_Early', 'amd_sim_data_Intermediate', 'amd_sim_data_Late-dry', 'amd_sim_data_Late-wet']
stage = stage_dirs[2] # Intermediate

#------------------------------------------------------------------------------------------------------------#
# Read input data

input_data = pd.read_csv(f'{input_dir}/{stage}/input_data.csv')
output_template = pd.read_csv(f'{input_dir}/{stage}/output_template.csv')
parameters = load_any(f'{input_dir}/{stage}/parameters.jsonc')    # dict. {'p': {}, 'age_weights': [], 'ages: []}
hierarchy = load_any(f'{input_dir}/{stage}/hierarchy.json')       # dict of lists. {'nodes': [], 'edges': []}
nodes_to_fit = load_any(f'{input_dir}/{stage}/nodes_to_fit.json') # LIST of strings

print(f'number of rows: {len(input_data)}')
print(f'number of unique location_id: {input_data["location_id"].nunique()}')
print('--------------------------------')

#------------------------------------------------------------------------------------------------------------#
# Create region_graph
nodes = hierarchy['nodes']
name_to_id = {} # warning: this does not handle duplicate names
id_to_name = {}

region_id_graph = nx.DiGraph()

for node in nodes:
  name_to_id[node[0]] = node[1]['location_id']
  id_to_name[node[1]['location_id']] = node[0]

  # create region_id_graph
  region_id_graph.add_node(node[1]['location_id'])

  my_id = node[1]['location_id']
  parent_id = node[1]['parent_id']
  if my_id != parent_id: # if my_id is not the root node
    region_id_graph.add_edge(parent_id, my_id)

print(f"number of nodes: {region_id_graph.number_of_nodes()}") 
print(f"number of edges: {region_id_graph.number_of_edges()}")

number of rows: 207
number of unique location_id: 18
--------------------------------
number of nodes: 233
number of edges: 232


In [76]:
id_to_name[422] # IDs are not incremental

'United States Virgin Islands'

In [77]:
# TASK 3: define "coords"

country_list = []
region_list = []
super_region_list = []

for node in hierarchy['nodes']:
  if node[1]['level'] == 3:
    country_list.append(node[1]['location_id'])
  elif node[1]['level'] == 2:
    region_list.append(node[1]['location_id'])
  elif node[1]['level'] == 1:
    super_region_list.append(node[1]['location_id'])
    
coords = {
    "country":      country_list,
    "region":       region_list,
    "super_region": super_region_list,
}

print(len(country_list))
print(len(region_list))
print(len(super_region_list))

204
21
7


In [78]:
# Utilizes id_to_name to print the name of the location
def describe():
        G = region_id_graph
        df = input_data
        for n in nx.dfs_postorder_nodes(G, 1):
            cnt = df['location_id'].eq(n).sum() + sum(G.nodes[c].get('cnt', 0) for c in G.successors(n))
            G.nodes[n]['cnt'] = int(cnt)
            G.nodes[n]['depth'] = nx.shortest_path_length(G, 1, n)
            
        for n in nx.dfs_preorder_nodes(G, 1):
            if G.nodes[n]['cnt'] > 0:
                print('  '*G.nodes[n]['depth'] + id_to_name[n], G.nodes[n]['cnt'])

# describe()

def keep():
    pass
    # Suggestion: filter input_data during "LOAD"

def filter_input_data_by_data_type(input_data: pd.DataFrame, data_type: str) -> pd.DataFrame:
        if not input_data.empty:
            return input_data[input_data['data_type'] == data_type]
        return input_data

# since our input_data only has 'p' data, this will return the same input_data
filter_input_data_by_data_type(input_data, 'p').head() 

,area,location_id,stage,stage_id,sex,sex_id,year_id,age_start,age_end,effective_sample_size,value,standard_error,x_sdi,x_tob,data_type,upper_ci,lower_ci,age_weights
0,Netherlands,89,Intermediate,5,Male,1,1990,55,64,1418.0,0.040903,0.005260,0.794612,0.434156,p,NaN,NaN,NaN
1,Netherlands,89,Intermediate,5,Female,2,1990,55,64,1802.0,0.033851,0.004260,0.794612,0.383810,p,NaN,NaN,NaN
2,Netherlands,89,Intermediate,5,Male,1,1990,65,74,1382.0,0.072359,0.006969,0.794612,0.434156,p,NaN,NaN,NaN
3,Netherlands,89,Intermediate,5,Female,2,1990,65,74,1865.0,0.036997,0.004371,0.794612,0.383810,p,NaN,NaN,NaN
4,Netherlands,89,Intermediate,5,Male,1,1990,75,84,796.0,0.103015,0.010774,0.794612,0.434156,p,NaN,NaN,NaN


In [79]:
# process.asr() | parameters
# ------------------------------------------------------------
data_type                = 'p'
reference_area           = 'Global'
reference_sex            = 'Both'
reference_year           = 'all'
mu_age                   = None
mu_age_parent            = None
sigma_age_parent         = None
rate_type                = 'neg_binom'
lower_bound              = None
interpolation_method     = 'linear'
include_covariates       = True
zero_re                  = False
# ------------------------------------------------------------

In [80]:
# process.asr() | local variables
# ------------------------------------------------------------
pm_model = pm.Model()

ages = np.array(parameters['ages'])
data = filter_input_data_by_data_type(input_data, data_type)
lb_data =filter_input_data_by_data_type(input_data, lower_bound) if lower_bound else None
params_of_data_type = parameters.get(data_type, {})

variables = {}
variables['data'] = data # TODO-variables

# check: mu_age_parent & sigma_age_parent
#  if either mu_age_parent or sigma_age_parent is NaN, set them to None
if (isinstance(mu_age_parent, np.ndarray) and np.isnan(mu_age_parent).any()) or \
    (isinstance(sigma_age_parent, np.ndarray) and np.isnan(sigma_age_parent).any()):

    mu_age_parent = None
    sigma_age_parent = None
# ------------------------------------------------------------

In [81]:
# process.asr() | [1] Prepare spline.spline (knots, smoothing)
# ------------------------------------------------------------
knots = np.array(params_of_data_type.get('parameter_age_mesh', np.arange(ages[0], ages[-1] + 1, 5)))


smooth_map = {'No Prior': np.inf, 'Slightly': 0.5, 'Moderately': 0.05, 'Very': 0.005}
smoothness_param = params_of_data_type.get('smoothness')

if isinstance(smoothness_param, dict): 
    amount = smoothness_param.get('amount')

    if isinstance(amount, (int, float)): # smoothness_param is dict, and amount is int or float
        smoothing = float(amount)
    else:                                # smoothness_param is dict, and amount may be string
        smoothing = smooth_map.get(amount, 0.0)

else:                                    # smoothness_param may be string
    smoothing = smooth_map.get(smoothness_param, 0.0)
# ------------------------------------------------------------

In [ ]:
# process.asr() | [2] Spline knots & smoothing


    smoothness_param = parameters.get('smoothness')
    


# "smoothness": {
#       "age_start": 2, // smoothing begins
#       "amount": "Slightly", // smoothing intensity
#       "age_end": 94, // smoothing ends
#     },

In [ ]:
# process.asr() | [3]

In [ ]:
# process.asr() | [4]

In [ ]:
### BELOW IS ORIGINAL CODE ###

In [3]:
# Level 3: country
countries = (
    hier[hier["Level"] == 3]
    .rename(columns={
        "Location ID": "loc3_id",
        "Location Name": "country",
        "Parent ID": "parent2_id"
    })
    [["loc3_id", "country", "parent2_id"]]
)

# Level 2: region
regions = (
    hier[hier["Level"] == 2]
    .rename(columns={
        "Location ID": "loc2_id",
        "Location Name": "region",
        "Parent ID": "parent1_id"
    })
    [["loc2_id", "region", "parent1_id"]]
)

# Level 1: super-region
superregs = (
    hier[hier["Level"] == 1]
    .rename(columns={
        "Location ID": "loc1_id",
        "Location Name": "super_region"
    })
    [["loc1_id", "super_region"]]
)


In [4]:
# 4.1 country 정보 붙이기
df = df.merge(countries, on="country", how="left")

# 4.2 region 정보 붙이기
df = df.merge(
    regions,
    left_on="parent2_id",
    right_on="loc2_id",
    how="left"
)

# 4.3 super-region 정보 붙이기
df = df.merge(
    superregs,
    left_on="parent1_id",
    right_on="loc1_id",
    how="left"
)

# 확인
df[["country","region","super_region"]].drop_duplicates().head()

,country,region,super_region
0,Sweden,Western Europe,High-income
1,United States of America,High-income North America,High-income
3,Israel,Western Europe,High-income
5,Switzerland,Western Europe,High-income
7,Canada,High-income North America,High-income


In [5]:

# 2) 매핑 딕셔너리
age_map  = {"Children":0, "Adult":1, "Both":2}
meth_map = {
    "Interview based":0,
    "Non-interview":1
}
prev_map = {
    "point prevalence":     0,
    "12-month prevalence":  1,
    "life-time prevalence": 2
}

# 3) long 포맷으로 전환
df_long = df.melt(
    id_vars=[
        "study",             
        "country",
        "region",
        "super_region",
        "Methods of questionnaire",
        "Age range",
        "study population"
    ],
    value_vars=[
        "point prevalence",
        "12-month prevalence",
        "life-time prevalence"
    ],
    var_name="prevalence_type",
    value_name="prevalence_value"
).dropna(subset=["prevalence_value"])

# 4) 코드 매핑
df_long["age_code"]  = df_long["Age range"].map(age_map)
df_long["meth_code"] = df_long["Methods of questionnaire"].map(meth_map)
df_long["prev_code"] = df_long["prevalence_type"].map(prev_map)

# 5) 최종 확인
result = df_long[[
    "study", "country", "region", "super_region",
    "age_code", "meth_code",
    "prev_code", "prevalence_value"
]]
print(result.head())

                     study                   country  \
0   Nilsson LV et al, 1984                    Sweden   
1     Kramer M et al, 1985  United States of America   
2    Flament M et al, 1990  United States of America   
3     Zohar AH et al, 1992                    Israel   
4  Reinherz HZ et al, 1993  United States of America   

                      region super_region  age_code  meth_code  prev_code  \
0             Western Europe  High-income         1          0          0   
1  High-income North America  High-income         1          0          0   
2  High-income North America  High-income         1          0          0   
3             Western Europe  High-income         0          0          0   
4  High-income North America  High-income         0          0          0   

   prevalence_value  
0              98.0  
1              22.0  
2              55.0  
3              36.0  
4              21.0  


In [6]:
# 6) 비율(p_hat) 및 로그 변환
#    prevalence_value: point/12-month/life-time 케이스 수
#    study population: 전체 표본 수

# 6.1 p_hat 계산
df_long["p_hat"] = df_long["prevalence_value"] / df_long["study population"]

# 6.2 로그 변환 (필요 시 0 회피를 위해 clamp)
eps = 1e-6
df_long["p_hat"] = df_long["p_hat"].clip(eps, 1 - eps)
df_long["log_p_hat"] = np.log(df_long["p_hat"])

In [ ]:
# 1) df_long 이용: prevalence_type 당 개수 세기
prev_count = df_long.groupby("study")["prevalence_type"] \
                    .transform("nunique")

# 2) 2개 이상이면 1, 아니면 0
df_long["multi_mask"] = (prev_count >= 2).astype(int)

# 3) 확인
df_long[["study","prevalence_type","multi_mask"]].drop_duplicates().head()

,study,prevalence_type,multi_mask
0,"Nilsson LV et al, 1984",point prevalence,0
1,"Kramer M et al, 1985",point prevalence,0
2,"Flament M et al, 1990",point prevalence,1
3,"Zohar AH et al, 1992",point prevalence,0
4,"Reinherz HZ et al, 1993",point prevalence,0


In [8]:
import pandas as pd

# 1) hierarchy 에서 좌표(리스트) 직접 추출
# level 1 → super_region (7개)
super_list = (
    hier[hier["Level"] == 1]
    ["Location Name"]
    .tolist()
)

# level 2 → region (21개)
region_list = (
    hier[hier["Level"] == 2]
    ["Location Name"]
    .tolist()
)

# level 3 → country (180+개)
country_list = (
    hier[hier["Level"] == 3]
    ["Location Name"]
    .tolist()
)

# 2) coords 정의 (unchanged)
coords = {
    "country":      country_list,
    "region":       region_list,
    "super_region": super_list,
    "age":          ["Children", "Adult", "Both"],
    "meth":         ["Interview based", "Non-interview"],
    "prev":         ["point prevalence", "12-month prevalence", "life-time prevalence"],
    "study":        df_long["study"].unique().tolist()
}

# 3) questionnaire method 컬럼명 변경 반영
#    이미 엑셀에서 "Methods of questionnaire" 아래 값을 "Interview based" / "Non-interview"로 바꿔 두셨다면
meth_map = {
    "Interview based":    0,
    "Non-interview":      1
}
df_long["meth_code"] = df_long["Methods of questionnaire"].map(meth_map)

# 4) index 배열 생성
country_idx = pd.Categorical(
    df_long["country"],
    categories=coords["country"]
).codes

region_idx = pd.Categorical(
    df_long["region"],
    categories=coords["region"]
).codes

super_idx = pd.Categorical(
    df_long["super_region"],
    categories=coords["super_region"]
).codes

age_idx   = df_long["age_code"].to_numpy()
meth_idx  = df_long["meth_code"].to_numpy()         
prev_idx  = df_long["prev_code"].to_numpy()

study_idx = pd.Categorical(
    df_long["study"],
    categories=coords["study"]
).codes

# 5) mask / 관측치 벡터 (unchanged)
multi_mask = df_long["multi_mask"].to_numpy()
y_obs      = df_long["log_p_hat"].to_numpy()


In [ ]:


with pm.Model(coords=coords) as model:

    # ─── Fixed effects ───
    β0    = pm.Normal("β0",    mu=0, sigma=10)
    β_age = pm.Normal("β_age", mu=0, sigma=5, dims="age")
    β_meth= pm.Normal("β_meth",mu=0, sigma=5, dims="meth")
    β_prev= pm.Normal("β_prev",mu=0, sigma=5, dims="prev")

    # ─── Non-centered random intercepts ───
    σ_sr = pm.HalfNormal("σ_sr", sigma=5)
    η_sr = pm.Normal("η_sr", mu=0, sigma=1, dims="super_region")
    u_sr = pm.Deterministic("u_sr", η_sr * σ_sr)

    σ_r  = pm.HalfNormal("σ_r", sigma=5)
    η_r  = pm.Normal("η_r",  mu=0, sigma=1, dims="region")
    u_r  = pm.Deterministic("u_r", η_r * σ_r)

    σ_c  = pm.HalfNormal("σ_c", sigma=5)
    η_c  = pm.Normal("η_c",  mu=0, sigma=1, dims="country")
    u_c  = pm.Deterministic("u_c", η_c * σ_c)

    # ─── Study-level random slope (non-centered) ───
    σ_ps= pm.HalfNormal("σ_prev_study", sigma=5)
    η_ps= pm.Normal("η_prev_study", mu=0, sigma=1, dims=("study","prev"))
    u_ps= pm.Deterministic("u_prev_study", η_ps * σ_ps)

    # ─── 선형 예측식 μ ───
    prev_effect = (β_prev[prev_idx] + u_ps[study_idx,prev_idx]) * multi_mask

    μ = (
        β0
        + β_age[age_idx]
        + β_meth[meth_idx]
        + prev_effect
        + u_sr[super_idx]
        + u_r[region_idx]
        + u_c[country_idx]
    )

    # ─── 관측모형 ───
    σ_y = pm.HalfNormal("σ_y", sigma=1)
    pm.Normal("y", mu=μ, sigma=σ_y, observed=y_obs)

    # ─── (옵션) ADVI 워밍업 — 빠르게 근사 posterior 생성 ───
    advi = pm.fit(method="advi", n=5000)

    # ─── NUTS 샘플링 ───
    idata = pm.sample(
        draws=1000, tune=500,
        chains=4, cores=4,
        target_accept=0.95,
        max_treedepth=15,
        progressbar="split"
    )

# ─── forest plot 으로 깔끔하게 시각화 ───
az.style.use("arviz-darkgrid")
az.plot_forest(
    idata,
    var_names=["β_age","β_meth","β_prev","σ_sr","σ_r","σ_c","σ_prev_study","σ_y"],
    combined=True,
    credible_interval=0.95,
    ridgeplot_overlap=0.5
)


Output()

Finished [100%]: Average Loss = 395.52
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [β0, β_age, β_meth, β_prev, σ_sr, η_sr, σ_r, η_r, σ_c, η_c, σ_prev_study, η_prev_study, σ_y]


Output()

Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 7188 seconds.


TypeError: plot_forest() got an unexpected keyword argument 'credible_interval'

In [12]:
import arviz as az

# idata는 이미 pm.sample()의 결과
summary_df = az.summary(
    idata,
    var_names=[
        "β0", "β_age", "β_meth", "β_prev",
        "σ_sr", "σ_r", "σ_c", "σ_prev_study", "σ_y"
    ],
    round_to=3,         # 소수점 자리수
    hdi_prob=0.95       # 95% 신뢰구간
)
print(summary_df)


                               mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  \
β0                           -3.537  4.085   -11.136      4.469      0.111   
β_age[Children]              -0.263  2.759    -5.733      4.756      0.080   
β_age[Adult]                 -0.284  2.752    -5.538      4.946      0.080   
β_age[Both]                  -0.183  2.763    -5.575      4.984      0.080   
β_meth[Interview based]       0.241  3.283    -6.431      6.356      0.080   
β_meth[Non-interview]        -0.924  3.280    -7.457      5.359      0.080   
β_prev[point prevalence]     -1.014  0.388    -1.760     -0.244      0.006   
β_prev[12-month prevalence]  -0.401  0.291    -0.957      0.177      0.006   
β_prev[life-time prevalence] -0.121  0.290    -0.680      0.434      0.006   
σ_sr                          0.916  0.575     0.008      1.955      0.017   
σ_r                           0.291  0.242     0.000      0.755      0.008   
σ_c                           0.455  0.218     0.011      0.816 

In [13]:


# 1) posterior를 (chain, draw) → sample 차원 하나로 합치기
post = idata.posterior.stack(sample=("chain","draw"))

# 2) coords에 저장된 리스트 불러오기
country_list      = coords["country"]       # Level-3 국가 이름 리스트
super_list        = coords["super_region"]  # Level-1 슈퍼리전
region_list       = coords["region"]        # Level-2 리전
age_list          = coords["age"]           # ["Children","Adult","Both"]
meth_list         = coords["meth"]          # ["Interview based","Non-interview"]
prev_list         = coords["prev"]          # ["point prevalence","12-month prevalence","life-time prevalence"]

# 3) 관심 범주의 posterior 샘플 꺼내기
β0_samps         = post["β0"].values                             # (samples,)
β_age_Both       = post["β_age"].sel(age="Both").values          # (samples,)
β_meth_IB        = post["β_meth"].sel(meth="Interview based").values  # (samples,)
β_prev_life      = post["β_prev"].sel(prev="life-time prevalence").values  # (samples,)

u_sr_samps       = post["u_sr"].values      # (super_region, samples)
u_r_samps        = post["u_r"].values       # (region, samples)
u_c_samps        = post["u_c"].values       # (country, samples)

# 4) country → region, super_region 매핑 (levels 파일에서 미리 만들어 두었다면 그것 사용)
#    여기선 hier 데이터프레임에 있는 Level, Location Name, Parent ID 이용
mapping = hier[["Level","Location ID","Location Name","Parent ID"]]
# Level3
lvl3 = mapping[mapping["Level"]==3].rename(columns={
    "Location Name":"country","Location ID":"loc3","Parent ID":"parent2"
})
# Level2
lvl2 = mapping[mapping["Level"]==2].rename(columns={
    "Location Name":"region","Location ID":"loc2","Parent ID":"parent1"
})
# Level1
lvl1 = mapping[mapping["Level"]==1].rename(columns={
    "Location Name":"super_region","Location ID":"loc1"
})

# merge to get each country’s region & super_region names
df_map = (
    lvl3[["country","loc3","parent2"]]
    .merge(lvl2[["loc2","region","parent1"]], left_on="parent2", right_on="loc2", how="left")
    .merge(lvl1[["loc1","super_region"]], left_on="parent1", right_on="loc1", how="left")
    [["country","region","super_region"]]
)

# 5) Monte Carlo 샘플마다 μ 계산 → p = exp(μ)
results = []
n_samps = β0_samps.shape[0]
for idx, row in df_map.iterrows():
    country    = row["country"]
    region     = row["region"]
    super_reg  = row["super_region"]
    i_sr = super_list.index(super_reg)
    i_r  = region_list.index(region)
    i_c  = country_list.index(country)

    # μ 샘플 벡터
    mu = (
        β0_samps
        + β_age_Both
        + β_meth_IB
        + β_prev_life
        + u_sr_samps[i_sr, :]
        + u_r_samps[i_r, :]
        + u_c_samps[i_c, :]
    )
    # prevalence 비율로 변환
    p = np.exp(mu)

    # median & 95% HDI
    med = np.median(p)
    hdi_lo, hdi_hi = az.hdi(p, hdi_prob=0.95)

    results.append({
        "country":             country,
        "region":              region,
        "super_region":        super_reg,
        "median_life_prev":    med,
        "hdi_2.5%":            hdi_lo,
        "hdi_97.5%":           hdi_hi
    })

df_out = pd.DataFrame(results)

# 6) 엑셀로 저장
output_path = "C:/Users/정이든/Desktop/연구실/9. prevalence of OCD/3차 와꾸/life_time_prevalence.xlsx"
df_out.to_excel(output_path, index=False)
print(f"Saved to {output_path}")


Saved to C:/Users/정이든/Desktop/연구실/9. prevalence of OCD/3차 와꾸/life_time_prevalence.xlsx
